In [4]:
import geopandas as geo
import pandas as pd
import numpy as np
from google.cloud import bigquery
import glob



In [5]:
date = "2020-09-08"

## Adds Palmira's New Cases

In [6]:
pattern = f'{date}/confirmados_*.shp'
filename = glob.glob(pattern)[0]
print(filename)
df = geo.read_file(filename)
df.head(10)

2020-09-08/confirmados_sep8_JF.shp


,OBJECTID,ID,nombre,ESTADO,ini_sin,num_ide,Caso,Institucio,Fecha_repo,geometry
0,1,2,Ingrig Johana Gomez,Recuperado,2020-03-09,1113656382,48,0,None,POINT (-76.31231 3.53602)
1,2,3,Samuel Chacon,Recuperado,2020-03-12,1114004654,2,0,None,POINT (-76.29874 3.50575)
2,3,4,Esperanza Cardenas,Recuperado,2020-03-13,29345055,3,0,None,POINT (-76.31004 3.54111)
3,4,5,Luz Dary Cordoba,Recuperado,2020-03-14,36951091,16,0,None,POINT (-76.29874 3.50575)
4,5,6,Esperanza Renjifo,Recuperado,2020-03-15,29500712,4,0,None,POINT (-76.27460 3.52889)
5,6,7,Carmen Caicedo,Recuperado,2020-03-17,31145339,14,0,None,POINT (-76.28539 3.53710)
6,7,8,Federico Henao,Recuperado,2020-03-16,1104824481,6,0,None,POINT (-76.31034 3.54232)
7,8,9,Ruth Salamanca,Recuperado,2020-03-16,31174164,29,0,None,POINT (-76.28264 3.52306)
8,9,10,Luis Angel BolaÃ±os,Recuperado,2020-03-21,76306282,28,0,None,POINT (-76.31231 3.53602)
9,10,11,Miriam Erazo,Recuperado,2020-03-11,25288907,1,0,None,POINT (-76.29957 3.50502)


In [7]:
df['lon'] = df.geometry.x
df['lat'] = df.geometry.y

In [9]:

df_confirmados = df[['OBJECTID','ini_sin','lon','lat']].rename(columns = {'OBJECTID':'case_id','ini_sin':'date_start_symtoms'})
df_confirmados['date_start_symtoms'] = pd.to_datetime(df_confirmados['date_start_symtoms'], errors = 'coerce')
df_confirmados.dropna(inplace = True)
df_confirmados.sort_values('date_start_symtoms')


,case_id,date_start_symtoms,lon,lat
815,816,2020-01-18,-76.301604,3.509680
0,1,2020-03-09,-76.312313,3.536019
9,10,2020-03-11,-76.299574,3.505019
1,2,2020-03-12,-76.298738,3.505747
2,3,2020-03-13,-76.310040,3.541110
...,...,...,...,...
1481,1482,2020-09-02,-76.345542,3.539936
1445,1446,2020-09-03,-76.293366,3.521252
1446,1447,2020-09-03,-76.281181,3.521007
1449,1450,2020-09-03,-76.317127,3.531906


In [10]:
#max_date = pd.to_datetime('2020-08-25')
max_date = None
if max_date is not None:
    df_confirmados = df_confirmados[df_confirmados.date_start_symtoms < max_date]
    df_confirmados.sort_values('date_start_symtoms')

In [11]:
df_confirmados.to_gbq(destination_table = 'positives.palmira',
                      project_id = 'grafos-alcaldia-bogota',
                 chunksize=None,
                 if_exists='replace'
                 )

1it [00:03,  3.49s/it]
